# Building Bi-LSTM to classify Tweets based on expressed sentiment 

In [ ]:
#import libraies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SimpleRNN, SpatialDropout1D
from keras.layers import  Dropout, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn import metrics
from google.colab import files
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt') 
from keras.metrics import Precision, Recall
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

In [ ]:
# Upload the CSV file
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [ ]:
data = pd.read_csv("Tweets.csv")

In [ ]:
data = data[['text', 'airline_sentiment']]
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [ ]:
data = data[data['airline_sentiment'] != 'neutral']
data.head()

# Tweets Pre-processing 

In [ ]:
#noisy removal
def clean_train_data(x):
    text = x
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) # remove square brackets
    text = re.sub(r'[^\w\s]','',text) # remove punctuation
    text = re.sub('\w*\d\w*', '', text) # remove words containing numbers
    text = re.sub(r'\s+', ' ', text, flags=re.I) # Substituting multiple spaces with single space
    text = re.sub('\n', '', text)
    return text

In [ ]:
data['text'] = data.text.apply(lambda x : clean_train_data(x))
data.head()

,text,airline_sentiment
0,virginamerica what dhepburn said,neutral
1,virginamerica plus youve added commercials to ...,positive
2,virginamerica i didnt today must mean i need t...,neutral
3,virginamerica its really aggressive to blast o...,negative
4,virginamerica and its a really big bad thing a...,negative


In [ ]:
#stopwords removal  
stop = set (stopwords.words ("english"))
def remove_stopwords (text): 
  text = [word.lower () for word in text.split() if word.lower() not in stop]
  return " ".join(text)

In [ ]:
data["text"] = data["text"].map(remove_stopwords)

In [ ]:
data["text"]

1        virginamerica plus youve added commercials exp...
3        virginamerica really aggressive blast obnoxiou...
4                       virginamerica really big bad thing
5        virginamerica seriously would pay flight seats...
6        virginamerica yes nearly every time fly vx ear...
                               ...                        
14633    americanair flight cancelled flightled leaving...
14634                         americanair right cue delays
14635       americanair thank got different flight chicago
14636    americanair leaving minutes late flight warnin...
14638    americanair money change flight dont answer ph...
Name: text, Length: 11541, dtype: object

In [ ]:
##input/output determination and dataset spiliting 
dataY = pd.get_dummies(data['airline_sentiment']).values
dX_train, dX_test, dY_train, dY_test = train_test_split(data['text'],dataY, test_size = 0.3, random_state = 42)

In [ ]:
#tweet words tokenization 
num_words= None
tokenizer = Tokenizer(num_words, split=' ') 
tokenizer.fit_on_texts(data['text'].values) 
X_train = tokenizer.texts_to_sequences(dX_train.values) 
X_train = pad_sequences(X_train)
X_test = tokenizer.texts_to_sequences(dX_test.values) 
X_test = pad_sequences(X_test)

In [ ]:
#to show how each word assigns to value after tokenizer 
word_index = tokenizer.word_index

In [ ]:
#word_index

In [ ]:
# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-08-09 17:15:01--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-08-09 17:15:01--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-08-09 17:15:01--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
embeddings_dictionary = dict()
embedding_dim = 300
glove_file = open('glove.6B.300d.txt')
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
#assign each word to its crossponding GloVe word vector 
vocab_len = len(word_index) + 1 
embedding_matrix = np.zeros((vocab_len, embedding_dim))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
##creating f1-score function to perform f1-score calculation 
def f1 (y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Building the Bi-LSTM model 

In [ ]:
lstm_out = 300
max_len=50
model = Sequential()
model.add(Embedding(vocab_len, 300, weights= [embedding_matrix], input_length=None, trainable=False))
model.add(SpatialDropout1D(0.5))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall(), f1])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 300)         3393000   
                                                                 
 spatial_dropout1d_2 (Spatia  (None, None, 300)        0         
 lDropout1D)                                                     
                                                                 
 bidirectional_2 (Bidirectio  (None, 600)              1442400   
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 2)                 1202      
                                                                 
Total params: 4,836,602
Trainable params: 1,443,602
Non-trainable params: 3,393,000
_________________________________________________________________


In [ ]:
print(X_train.shape,dY_train.shape)
print(X_test.shape,dY_test.shape)

(8078, 22) (8078, 2)
(3463, 20) (3463, 2)


In [ ]:
#model training 
batch_size = 32
model.fit(X_train, dY_train, epochs = 10, batch_size=batch_size, verbose = 2)

In [ ]:
#model testing 
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = dY_test[-validation_size:]
X_test = X_test[:-validation_size]
dY_test = dY_test[:-validation_size]
score,acc, Precision, Recall, f1= model.evaluate(X_test, dY_test) #verbose = 2, batch_size = batch_size
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))
print("precision: %.2f" % (Precision))
print("recall: %.2f" % (Recall))
print("f1: %.2f" % (f1))

15/15 [==============================] - 0s 22ms/step - loss: 0.2229 - accuracy: 0.9266 - precision_1: 0.9266 - recall_1: 0.9266 - f1: 0.9268
score: 0.22
acc: 0.93
precision: 0.93
recall: 0.93
f1: 0.93


In [ ]:
from sklearn.metrics import classification_report

history = model.fit(X_test, dY_test, verbose = 2, batch_size = batch_size, shuffle=True)
pred = model.predict(X_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(dY_test, axis=1), predicted)
print(report)

62/62 - 15s - loss: 0.2473 - accuracy: 0.9037 - precision: 0.9037 - recall: 0.9037 - f1: 0.9038 - 15s/epoch - 241ms/step
62/62 [==============================] - 1s 17ms/step
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      1581
           1       0.86      0.82      0.84       382

    accuracy                           0.94      1963
   macro avg       0.91      0.89      0.90      1963
weighted avg       0.94      0.94      0.94      1963



In [ ]:
# Predictions; Applying the trained model to new tweets
predictions = model.predict(X_test)
predictions = np.round(predictions).astype(int)
predictions=np.argmax(predictions, axis=1, out=None)

In [ ]:
df_test = pd.DataFrame(dX_test[:-validation_size]).reindex()
df_test["target"] = predictions

In [ ]:
# Creating submission file 
#submission = pd.DataFrame( data['text'])
df_test.to_csv('Tweets.csv', index=False)
df_test.head(20)

,text,target
11825,usairways charged flight cancelled flightled u...,0
8105,jetblue great flight great view,1
1279,united theyre actually gate agent rude standin...,0
14343,americanair worries called back hrs late fligh...,0
3829,united thank one months ago none weird dont cl...,1
2590,united brothers luggage lost copa airlines fli...,0
11912,americanair great thanks followed,1
11544,usairways work hard making sure things flow sm...,0
3864,united told lostyou guys dont know,0
7940,jetblue bein calling jetblue respond leave num...,0
